In [ ]:
import os
import re
import edgar
import subprocess
import pandas as pd

In [ ]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

In [ ]:
# unload files into 'unload' folder
folder = 'unload'
startYear = '2017'
cmd = f'python run.py -y {startYear} -d {folder}'

process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
output, error = process.communicate()
print('\n'.join(output.decode("utf-8").split(' - ')))

In [ ]:
from func_tickSearch import cikftick
cikftick(['AAPL'])

In [ ]:
# <<< File Lookup Pattern >>>
# pattern = r'^\d{4}-QTR\d\.tsv'
def edgarPattern(year=None, qtr=None):
    pattern = r'^\d{4}-QTR\d\.tsv'
    if year is not None:
        if isinstance(year, int):
            year = str(year)
        assert (re.match(r'\d{4}', year) is not None) & (len(year) == 4), (
            "'year' must be 4-digit integer or string")
    if qtr is not None:
        assert (int(qtr) > 0) & (int(qtr) < 5), (
            "'qtr' must be 1-digit integer between 1 and 4")
    
    if year is not None:
        pattern = pattern.replace('\d{4}', str(year))
    if qtr is not None:
        pattern = pattern.replace('QTR\d', 'QTR' + str(qtr))
    return pattern

In [ ]:
# <<< Merging >>>
# merge unloaded files into master index in 'mstrIdx' folder
mstr_folder = 'mstrIdx'
os.makedirs(mstr_folder, exist_ok=True)
# cmd = f'cat {folder}/*.tsv > {mstr_folder}/master.tsv'

# file_lst = [x for x in os.listdir(folder) if x[-4:]=='.tsv']
file_lst = [x for x in os.listdir(folder) if re.match(r'^.+\.tsv$', x)]
with open(f'{mstr_folder}/master.tsv', 'w') as f:
    for fname in file_lst:
        with open(f'{folder}/{fname}') as infile:
            for line in infile:
                f.write(line)
        print(fname)

In [ ]:
# Select security and find its index
code = '1000045'

idx_list = []
file_lst = [x for x in os.listdir(folder) if re.match(r'^.+\.tsv$', x)]
for file in file_lst:
    with open(f'{folder}/{file}') as f:
        for line in f:
            if code.lstrip('0') in line:
                idx_list.append(line)
idx_list = [x.split('|') for x in idx_list]

In [ ]:
idxDF = pd.DataFrame(idx_list, columns=['Code', 'Name', 'fileType', 'reportDate', 'raw_path', 'url'])
print(idxDF.head())

In [ ]:
idxDF['fileType'].unique()

#### 10-K

In [ ]:
filing_10k_lst = idxDF.loc[idxDF['fileType']=='10-K'].reset_index()
print(filing_10k_lst)

In [ ]:
tmp_lst = filing_10k_lst.T.to_dict()
for i in tmp_lst:
    init_ = 'https://www.sec.gov/Archives'
    url_path = '/'.join([init_, tmp_lst[i]['raw_path']])
    date_ = tmp_lst[i]['reportDate'].replace('-', '')
    data_root = 'data/' + str(tmp_lst[i]['fileType'])
    data_folder = '/'.join([data_root, code])
    os.makedirs(data_folder, exist_ok=True)
    cmd = f'curl {url_path} --output {data_folder}/{code}_{date_}.html'
    print(cmd)

    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()

#### 10-Q

In [ ]:
filing_10q_lst = idxDF.loc[idxDF['fileType']=='10-Q'].reset_index()
print(filing_10q_lst)

In [ ]:
tmp_lst = filing_10q_lst.T.to_dict()
for i in tmp_lst:
    init_ = 'https://www.sec.gov/Archives'
    url_path = '/'.join([init_, tmp_lst[i]['raw_path']])
    date_ = tmp_lst[i]['reportDate'].replace('-', '')
    data_root = 'data/' + str(tmp_lst[i]['fileType'])
    data_folder = '/'.join([data_root, code])
    os.makedirs(data_folder, exist_ok=True)
    cmd = f'curl {url_path} --output {data_folder}/{code}_{date_}.html'
    print(cmd)

    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()

#### 8-K

In [ ]:
filing_8k_lst = idxDF.loc[idxDF['fileType']=='8-K'].reset_index()
print(filing_8k_lst)

In [ ]:
tmp_lst = filing_8k_lst.T.to_dict()
for i in tmp_lst:
    init_ = 'https://www.sec.gov/Archives'
    url_path = '/'.join([init_, tmp_lst[i]['raw_path']])
    date_ = tmp_lst[i]['reportDate'].replace('-', '')
    data_root = 'data/' + str(tmp_lst[i]['fileType'])
    data_folder = '/'.join([data_root, code])
    os.makedirs(data_folder, exist_ok=True)
    cmd = f'curl {url_path} --output {data_folder}/{code}_{date_}.html'
    print(cmd)

    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()